# Data Aggregation and Group Operations

*** 

Author: Jiacheng

Date: 2019-05-19

***

In [1]:
import numpy as np
import pandas as pd

---

## 1. GroupBy Mechanics
## GroupBy 机制

* ### 1. 入门

In [3]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-1.367304,0.180706
1,a,two,-2.706003,0.310918
2,b,one,-1.429791,-0.623735
3,b,two,-1.864855,0.546224
4,a,one,0.512964,-0.252997


In [4]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [5]:
grouped.mean()

key1
a   -1.186781
b   -1.647323
Name: data1, dtype: float64

In [6]:
# 多重分组
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.427170
      two    -2.706003
b     one    -1.429791
      two    -1.864855
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,one,two
key1,,
a,-0.427170,-2.706003
b,-1.429791,-1.864855


In [8]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean().unstack()

,2005,2006
California,-2.706003,-1.429791
Ohio,-1.616079,0.512964


In [9]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.427170 -0.036146
     two  -2.706003  0.310918
b    one  -1.429791 -0.623735
     two  -1.864855  0.546224

In [10]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

* ### 2. 对分组进行迭代

In [11]:
# 迭代产生一组二元元组
for name, group in df.groupby('key1'):
     print(name)
     print(group)

a
  key1 key2     data1     data2
0    a  one -1.367304  0.180706
1    a  two -2.706003  0.310918
4    a  one  0.512964 -0.252997
b
  key1 key2     data1     data2
2    b  one -1.429791 -0.623735
3    b  two -1.864855  0.546224


In [12]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -1.367304  0.180706
4    a  one  0.512964 -0.252997
('a', 'two')
  key1 key2     data1     data2
1    a  two -2.706003  0.310918
('b', 'one')
  key1 key2     data1     data2
2    b  one -1.429791 -0.623735
('b', 'two')
  key1 key2     data1     data2
3    b  two -1.864855  0.546224


In [13]:
pieces = dict(list(df.groupby('key1')))

In [14]:
pieces

{'a':   key1 key2     data1     data2
 0    a  one -1.367304  0.180706
 1    a  two -2.706003  0.310918
 4    a  one  0.512964 -0.252997, 'b':   key1 key2     data1     data2
 2    b  one -1.429791 -0.623735
 3    b  two -1.864855  0.546224}

In [15]:
pieces['a']

,key1,key2,data1,data2
0,a,one,-1.367304,0.180706
1,a,two,-2.706003,0.310918
4,a,one,0.512964,-0.252997


In [16]:
# 将根据dtypes 进行分组
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [17]:
grouped = df.groupby(df.dtypes, axis=1)

In [18]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -1.367304  0.180706
1 -2.706003  0.310918
2 -1.429791 -0.623735
3 -1.864855  0.546224
4  0.512964 -0.252997
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


* ### 3. 选取一列或列的子集

In [19]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.036146
     two   0.310918
b    one  -0.623735
     two   0.546224

In [20]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [21]:
s_grouped.mean()

key1  key2
a     one    -0.036146
      two     0.310918
b     one    -0.623735
      two     0.546224
Name: data2, dtype: float64

* ### 4. 通过字典或Series进行分组

In [23]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1,2]] = np.nan
people


,a,b,c,d,e
Joe,1.409747,-0.156562,-0.838291,-0.760133,1.550084
Steve,-0.398073,1.299256,0.669995,-1.460579,-0.606086
Wes,0.569048,NaN,NaN,-0.421104,0.711745
Jim,-0.714494,-0.035402,-0.475010,0.057012,1.087622
Travis,0.664716,-0.167047,0.671265,0.237326,-1.237347


In [24]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-1.598423,2.803270
Steve,-0.790583,0.295097
Wes,-0.421104,1.280794
Jim,-0.417998,0.337727
Travis,0.908590,-0.739678


In [25]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [26]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


* ### 5. 通过函数进行分组